# Part 2: boost-histogram plans and Hist

## Boost-histogram numbafication plans

In [1]:
# todo

## Hist

Hist extends boost-histogram with:
* Named axes and labels
* Fancy Jupyter reprs
* UHI+: faster, easier to type indexing additions
* QuickConstruct: a system to reduce the typing when making histograms
* Other useful shortcuts, like density, pandas support, profiles from existing histograms, and more!

### Name shortcuts of Hist

Hist allows names for Boost-histograms axes, the names are unique identifiers in a histogram which are used to support some useful features such as `.fill()` and `.project()`. Specially, hist designs `NamedHist` to provide name shortcuts for the histograms that contain named axes.

In [2]:
import numpy as np
import boost_histogram as bh

from hist import axis, Hist, NamedHist

# named axes
reg_axis = axis.Regular(10, -3, 3, overflow=False, underflow=False, name="X", label="x [unit]")
var_axis = axis.Variable(range(-5, 6), name="Y", label="y [unit]")
int_axis = axis.Integer(-3, 3, overflow=True, underflow=True, name="Z", label="z [units]")

In [3]:
# histograms with named axes
unnamed_hist = Hist(reg_axis, var_axis, int_axis)
named_hist = NamedHist(reg_axis, var_axis, int_axis)

print("Name of axis 0: \t" + named_hist.axes[0].name + ";")
print("Label of axis 1: \t" + named_hist.axes[1].label + ".")

Name of axis 0: 	X;
Label of axis 1: 	y [unit].


In [4]:
# name shortcuts
unnamed_hist = unnamed_hist.fill(np.random.randn(100), np.random.randn(100), np.random.randn(100))
unnamed_hist_2d = unnamed_hist.project(0, 1)
named_hist = named_hist.fill(X=np.random.randn(100), Y=np.random.randn(100), Z=np.random.randn(100))
named_hist_2d = named_hist.project("X", "Y")

### Repr

Hist has custom reprs when displaying in a Jupyter and supports dark mode.

In [5]:
named_hist_2d.project("X")

NamedHist(Regular(10, -3, 3, underflow=False, overflow=False, name='X', label='x [unit]'), storage=Double()) # Sum: 99.0

In [6]:
named_hist_2d

NamedHist(
  Regular(10, -3, 3, underflow=False, overflow=False, name='X', label='x [unit]'),
  Variable([-5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5], name='Y', label='y [unit]'),
  storage=Double()) # Sum: 99.0

In [7]:
named_hist

NamedHist(
  Regular(10, -3, 3, underflow=False, overflow=False, name='X', label='x [unit]'),
  Variable([-5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5], name='Y', label='y [unit]'),
  Integer(-3, 3, name='Z', label='z [units]'),
  storage=Double()) # Sum: 99.0

Besides the fancy repr, the users can explictly see the data by `.plot()` (using mplhep in the backend) and `.show()` (using histoprint).

### UHI

Uniform Histogram Indexing (UHI) is one of the most important features of hist, which provides HEP users with handy accessing shortcuts. For example, to access the centroid element of a 2d-histogram, we can:

In [8]:
print(unnamed_hist_2d[5, 5])
print(unnamed_hist_2d[{0: 5, 1: 5}])
print(unnamed_hist_2d[{"X": 5, "Y": 5}])
print(unnamed_hist_2d[{"X": bh.loc(0), "Y": bh.loc(0)}])
print(unnamed_hist_2d[-1+.8j, 1-.5j])

5.0
5.0
5.0
5.0
5.0


UHI also supports rebin for histograms.

In [9]:
named_hist_2d[:, 1-.5j]

NamedHist(Regular(10, -3, 3, underflow=False, overflow=False, name='X', label='x [unit]'), storage=Double()) # Sum: 35.0

In [10]:
named_hist_2d[0:10:2j, 1-.5j]

NamedHist(Regular(5, -3, 3, underflow=False, overflow=False, name='X', label='x [unit]'), storage=Double()) # Sum: 35.0

### Quick Construct

Besides the standard construction of boost-histogram, hist provides quick construct for HEP users.

In [11]:
unnamed_hist = (
    Hist.new.Reg(50, -3, 3, flow=False)
    .Var(range(-25, 30))
    .Int(-3, 3, flow=True)
    .Double()
)

named_hist = (
    Hist.new.Reg(50, -3, 3, flow=False, name="X", label="x [unit]")
    .Var(range(-25, 30), name="Y", label="y [unit]")
    .Int(-3, 3, flow=True, name="Z", label="z [units]")
    .Double()
)

In [12]:
unnamed_hist.fill(np.random.randn(100), 5*np.random.randn(100), np.random.randn(100)).project(0, 1)

Hist(
  Regular(50, -3, 3, label='Axis 0'),
  Variable(array([-25., -24., -23., -22., -21., -20., -19., -18., -17., -16., -15.,
       -14., -13., -12., -11., -10.,  -9.,  -8.,  -7.,  -6.,  -5.,  -4.,
        -3.,  -2.,  -1.,   0.,   1.,   2.,   3.,   4.,   5.,   6.,   7.,
         8.,   9.,  10.,  11.,  12.,  13.,  14.,  15.,  16.,  17.,  18.,
        19.,  20.,  21.,  22.,  23.,  24.,  25.,  26.,  27.,  28.,  29.]), label='Axis 1'),
  storage=Double()) # Sum: 100.0

In [13]:
named_hist.fill(X=np.random.randn(100), Y=5*np.random.randn(100), Z=np.random.randn(100)).project("X", "Y")

Hist(
  Regular(50, -3, 3, name='X', label='x [unit]'),
  Variable(array([-25., -24., -23., -22., -21., -20., -19., -18., -17., -16., -15.,
       -14., -13., -12., -11., -10.,  -9.,  -8.,  -7.,  -6.,  -5.,  -4.,
        -3.,  -2.,  -1.,   0.,   1.,   2.,   3.,   4.,   5.,   6.,   7.,
         8.,   9.,  10.,  11.,  12.,  13.,  14.,  15.,  16.,  17.,  18.,
        19.,  20.,  21.,  22.,  23.,  24.,  25.,  26.,  27.,  28.,  29.]), name='Y', label='y [unit]'),
  storage=Double()) # Sum: 100.0

### Other Shortcuts

Hist also provides some useful shortcuts for HEP analysis. For example, we can get the density of an existing histogram via `.density()`.

In [18]:
named_hist.project("X")[25:30].density()

array([2.88461538, 0.96153846, 0.96153846, 1.6025641 , 1.92307692])

In [20]:
import pandas as pd

data = pd.read_csv(
    "results-20210227-133657 - results-20210227-133657.csv",
    usecols=("cpu", "num_downloads", "python_version", "pip_version", "glibc_version", "policy"),
    converters={
        "python_version": str,
        "pip_version": lambda x: int(x.split(".")[0]),
        "glibc_version": lambda x: int(float(x.split("-")[0]) % 1 * 100),
    },
)

FileNotFoundError: [Errno 2] No such file or directory: 'results-20210227-133657 - results-20210227-133657.csv'